In [2]:
from selenium import webdriver

from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC

import time

from concurrent.futures import ThreadPoolExecutor

In [5]:
start_year = 2001
end_year = 2021

In [9]:
list(range(start_year, end_year + 1))

[2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021]

In [10]:
chrome_options = ChromeOptions()
# service = ChromeService(executable_path="C:/Executables/chromedriver-win64/chromedriver.exe")
# chrome_options.add_experimental_option('detach', True)
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

In [11]:
driver.get('https://fred.stlouisfed.org/release/tables?rid=118&eid=259194&od=2001-01-01#')
time.sleep(5)

In [13]:
table_rows = driver.find_elements(by=By.CSS_SELECTOR, value="tr[data-tt-id]")
table_rows

[<selenium.webdriver.remote.webelement.WebElement (session="08a1f780a15b611c5a9f13a69078891f", element="f.A155D5A2DB37C1081551FB541363500F.d.13637C991A17A6CDDEED9B6DBDDA33BD.e.229")>,
 <selenium.webdriver.remote.webelement.WebElement (session="08a1f780a15b611c5a9f13a69078891f", element="f.A155D5A2DB37C1081551FB541363500F.d.13637C991A17A6CDDEED9B6DBDDA33BD.e.101")>,
 <selenium.webdriver.remote.webelement.WebElement (session="08a1f780a15b611c5a9f13a69078891f", element="f.A155D5A2DB37C1081551FB541363500F.d.13637C991A17A6CDDEED9B6DBDDA33BD.e.111")>,
 <selenium.webdriver.remote.webelement.WebElement (session="08a1f780a15b611c5a9f13a69078891f", element="f.A155D5A2DB37C1081551FB541363500F.d.13637C991A17A6CDDEED9B6DBDDA33BD.e.121")>,
 <selenium.webdriver.remote.webelement.WebElement (session="08a1f780a15b611c5a9f13a69078891f", element="f.A155D5A2DB37C1081551FB541363500F.d.13637C991A17A6CDDEED9B6DBDDA33BD.e.131")>,
 <selenium.webdriver.remote.webelement.WebElement (session="08a1f780a15b611c5a9f

In [16]:
len(table_rows)

51

In [19]:
table_rows[0].find_element(by=By.CSS_SELECTOR, value='th.fred-rls-elm-nm-cntnr').text

'  Alabama'

In [20]:
table_rows[0].find_element(by=By.CSS_SELECTOR, value='td.fred-rls-elm-vl-td').text

'4,467.634'

In [32]:
def helper(element):
    state = element.find_element(by=By.CSS_SELECTOR, value='th.fred-rls-elm-nm-cntnr').text
    state = state.strip()
    population = element.find_element(by=By.CSS_SELECTOR, value='td.fred-rls-elm-vl-td').text
    population = population.strip()

    return state, population
with ThreadPoolExecutor() as exe:
    state_populations = dict(exe.map(helper, table_rows))

In [33]:
state_populations

{'Alabama': '4,467.634',
 'Alaska': '633.714',
 'Arizona': '5,273.477',
 'Arkansas': '2,691.571',
 'California': '34,479.458',
 'Colorado': '4,425.687',
 'Connecticut': '3,432.835',
 'Delaware': '795.699',
 'District of Columbia': '574.504',
 'Florida': '16,356.966',
 'Georgia': '8,377.038',
 'Hawaii': '1,225.948',
 'Idaho': '1,319.962',
 'Illinois': '12,488.445',
 'Indiana': '6,127.760',
 'Iowa': '2,931.997',
 'Kansas': '2,702.162',
 'Kentucky': '4,068.132',
 'Louisiana': '4,477.875',
 'Maine': '1,285.692',
 'Maryland': '5,374.691',
 'Massachusetts': '6,397.634',
 'Michigan': '9,991.120',
 'Minnesota': '4,982.796',
 'Mississippi': '2,852.994',
 'Missouri': '5,641.142',
 'Montana': '906.961',
 'Nebraska': '1,719.836',
 'Nevada': '2,098.399',
 'New Hampshire': '1,255.517',
 'New Jersey': '8,492.671',
 'New Mexico': '1,831.690',
 'New York': '19,082.838',
 'North Carolina': '8,210.122',
 'North Dakota': '639.062',
 'Ohio': '11,387.404',
 'Oklahoma': '3,467.100',
 'Oregon': '3,467.937',
 

In [ ]:
import pandas as pd

In [44]:
def extract_populations(start_year: int, end_year: int):
    years = []
    for year in list(range(start_year, end_year + 1)):
        # visit page per year
        print(f'processing year {year}')
        driver.get(f'https://fred.stlouisfed.org/release/tables?rid=118&eid=259194&od={year}-01-01#')
        
        # sleep for a moment
        time.sleep(5)

        # extract all table rows in page
        table_rows = driver.find_elements(by=By.CSS_SELECTOR, value="tr[data-tt-id]")

        # concurrently extract all states populations
        def helper(element):
            state = element.find_element(by=By.CSS_SELECTOR, value='th.fred-rls-elm-nm-cntnr').text
            state = state.strip()
            population = element.find_element(by=By.CSS_SELECTOR, value='td.fred-rls-elm-vl-td').text
            population = population.strip()

            return state, population
        
        with ThreadPoolExecutor() as exe:
            # state populations will be a dictionary
            state_populations = list(exe.map(helper, table_rows))

        states, populations = zip(*state_populations)
        yearly_df = pd.DataFrame({"state": states, "population": populations})
        yearly_df["year"] = year
        years.append(yearly_df)

    # return all years containing all states populations
    return years

In [45]:
years = extract_populations(start_year=start_year, end_year=end_year)
years

processing year 2001
processing year 2002
processing year 2003
processing year 2004
processing year 2005
processing year 2006
processing year 2007
processing year 2008
processing year 2009
processing year 2010
processing year 2011
processing year 2012
processing year 2013
processing year 2014
processing year 2015
processing year 2016
processing year 2017
processing year 2018
processing year 2019
processing year 2020
processing year 2021


[                   state  population  year
 0                Alabama   4,467.634  2001
 1                 Alaska     633.714  2001
 2                Arizona   5,273.477  2001
 3               Arkansas   2,691.571  2001
 4             California  34,479.458  2001
 5               Colorado   4,425.687  2001
 6            Connecticut   3,432.835  2001
 7               Delaware     795.699  2001
 8   District of Columbia     574.504  2001
 9                Florida  16,356.966  2001
 10               Georgia   8,377.038  2001
 11                Hawaii   1,225.948  2001
 12                 Idaho   1,319.962  2001
 13              Illinois  12,488.445  2001
 14               Indiana   6,127.760  2001
 15                  Iowa   2,931.997  2001
 16                Kansas   2,702.162  2001
 17              Kentucky   4,068.132  2001
 18             Louisiana   4,477.875  2001
 19                 Maine   1,285.692  2001
 20              Maryland   5,374.691  2001
 21         Massachusetts   6,39

# Export extracted data to dataframe

data will be arranged in this manner:
```
| year | state | population |
| 2001 | Alabama | 4,480.089 |
| 2001 | Alaska | 642.337 |

In [47]:
final = pd.concat(years, axis=0, ignore_index=True)
final

,state,population,year
0,Alabama,"4,467.634",2001
1,Alaska,633.714,2001
2,Arizona,"5,273.477",2001
3,Arkansas,"2,691.571",2001
4,California,"34,479.458",2001
...,...,...,...
1066,Virginia,"8,658.910",2021
1067,Washington,"7,743.760",2021
1068,West Virginia,"1,785.618",2021
1069,Wisconsin,"5,881.608",2021


In [48]:
final.to_csv(f'./data/us_populations_per_state_{start_year}_to_{end_year}.csv')